### 1.- Instalar Dependencias

In [125]:
import agentpy as ap
import matplotlib.pyplot as plt
import numpy as np

# Visualization
import seaborn as sns
import IPython

### 2.- Definición de Agentes

In [126]:
# Complete next_random_position
# Change clenaing room to add more agents
# Don't use next_position
class CleaningRobot(ap.Agent):

    #Constructor
    def setup(self):
      
      # direction atributes
      self.dx = 1
      self.dy = 0



      pass

    #Reglas de desplazamiento
    def setup_rules(self, xlim, ylim):

        # Save displacement limits
        self.xlim = xlim
        self.ylim = ylim


        pass

    #Obtener posicion
    def get_position(self, world):

        # Get agent position in the world
        pos = world.positions[self]
        return pos

    #Obtener estado siguiente
    def get_next_state(self, world):

        # get agent position
        position = self.get_position(world)

        # Verify if the next position is out of the limits
        if position[0] + self.dy < 0 or position[1] + self.dx < 0 or position[0] + self.dy > self.ylim or position[1] + self.dx > self.xlim:
            return [-1]
        else:
            
            # Read next position state
            value = world.agents[(position[0] + self.dy, position[1] + self.dx)].condition

            # convert iteration to state
            values = []
            for i in value:
                values.append(i)

            return values
            

    #Moverse de forma aleatoria
    def next_random_position(self):

        pass

    #Moverse a siguiente posicion hacia la derecha
    def next_position(self, world):

        # get agent position
        position = self.get_position(world)

        # Get next status
        value = self.get_next_state(world)

        # Move agent
        if value[0] != -1 and value[0] != 2 and value[0] != 3: # Hole, obstacle or robot
            world.move_to(self, (position[0] + self.dy, position[1] + self.dx))
        else:
            # Conditions table
            # A) Right
            if self.dx == 1 and self.dy == 0:

                # To the bottom
                self.dx = 0
                self.dy = 1
            # B) Bottom
            elif self.dx == 0 and self.dy == 1:

                # To the left
                self.dx = -1
                self.dy = 0
            # C) Left
            elif self.dx == -1 and self.dy == 0:

                # To the top
                self.dx = 0
                self.dy = -1
            # D) Top
            elif self.dx == 0 and self.dy == -1:

                # To the right
                self.dx = 1
                self.dy = 0
        
        pass

    #Limpiar area
    def clear_area(self, world):

        # Clean actual position
        position = self.get_position(world)
        world.agents[position].condition = 0

        # Get environment positions
        environment = world.neighbors(self, 1)

        # Clean around positions
        for floor in environment:
            if floor.condition == 1:
                floor.condition = 0 # Change to clean


        pass

### 3.- Creación del Ambiente

In [127]:
class CleaningRoom(ap.Model):

    #Metodo de configuracion
    def setup(self):

        # Create floor agents
        n_floor = int(self.p['dim'] * self.p['dim'])
        self.floor_agents = ap.AgentList(self, n_floor)

        # Create cleaning robot
        self.robot_agents = ap.AgentList(self, self.p['num_robots'], CleaningRobot)

        # Create environment
        self.room = ap.Grid(self, [self.p['dim']]*2, track_empty=True)
        
        # Add agents to the environment
        self.room.add_agents(self.robot_agents, random=True)
        self.room.add_agents(self.floor_agents)

        # Initialize floor status
        # 0: Clean, 1: Dirty, 2: Obstacle
        self.room.agents.condition = 0

        # Set dirty spots
        dirty_index = np.random.randint(len(self.floor_agents), 
                                        size=(int(self.p['den_dirt']*self.p['dim']**2)))
        
        for i in range(len(dirty_index)):
            self.floor_agents[dirty_index[i]].condition = 1


        # Set obstacles
        obstacle_index = np.random.randint(len(self.floor_agents), 
                                        size=(int(self.p['den_obs']*self.p['dim']**2)))
        
        for i in range(len(obstacle_index)):
            self.floor_agents[obstacle_index[i]].condition = 2

        # Assign state to robot
        self.robot_agents[0].condition = 3
        self.room.agents[self.room.positions[self.robot_agents[0]]].condition = 3
                
        # Initialize robot atributtes
        self.robot_agents[0].setup_rules(self.p['dim']-1, self.p['dim']-1)

    #Iteracion
    def step(self):

        # Clean area
        self.robot_agents[0].clear_area(self.room)

        # Move to next position
        self.robot_agents[0].next_position(self.room)

        # Get actual position
        position = self.room.positions[self.robot_agents[0]]

        # Check robot possition
        self.room.agents[position].condition = 3




        pass

    #Finalizar simulacion
    def end(self):

        pass

### 4.- Simulación

Funciones Auxiliares

In [128]:
#Convertor de RGB a HEX
def rgb2hex(r, g, b):

  var = '#%02x%02x%02x' % (r, g, b)

  return var

In [129]:
# Create single-run animation with custom colors
def animation_plot(model, ax):
    attr_grid = model.room.attr_grid('condition')
    ap.gridplot(attr_grid, ax=ax, color_dict=color_array, convert=True)

Parámetros de Simulación

In [130]:
#Crear paleta de colores
color_array = {0:rgb2hex(255, 255, 255), 1:rgb2hex(200, 200, 200), 2:rgb2hex(0, 0, 0), 3:rgb2hex(0, 255, 0), None:rgb2hex(255, 255, 255)}

In [131]:
# Inicializar parametros
parameters = {
    'dim': 20, # Scenario dimensions
    'den_obs': 0.1, # % Density of obstacles
    'den_dirt': 0.4, # % Density of dirt
    'num_robots': 1, # Number of robots
}

In [132]:
#Crear instancia
little_floor = CleaningRoom(parameters)


In [133]:
#Correr Simulacion
fig, ax = plt.subplots()
animation = ap.animate(little_floor, fig, ax, animation_plot, steps = 50)
IPython.display.HTML(animation.to_jshtml(fps=15))